# CaDRReS-Sc Training
This notebook show an example of how to fit CaDRReS-Sc model on a training set (e.g GDSC or any pharmacogenomic experiment you might have) and save its final version to make predictions on new data. For the detail of the new objective function, please refer to our manuscript.

In [93]:
import sys, os, pickle
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)
from collections import Counter

import importlib
from ipywidgets import widgets
import warnings
warnings.filterwarnings('ignore')

scriptpath = '..'
sys.path.append(os.path.abspath(scriptpath))

from cadrres_sc import pp, model, evaluation, utility
importlib.reload(model)

<module 'cadrres_sc.model' from 'c:\\Users\\carey\\Desktop\\CaDRReS-Sc\\cadrres_sc\\model.py'>

# Read data
In this step we expect a dataset holding gene expression and screened drug responses. In the following example, we load this information from the GDSC dataset.  

## Read cell line info

- Cell line tissue info
- Observed drug response IC50 to be used as ground truth

In [119]:
tissue_sample_df = pd.read_csv('../data/GDSC/GDSC_tissue_info.csv', index_col=0)
tissue_sample_df.index = tissue_sample_df.index.astype(str)

cell_line_obs_df = pd.read_csv('../data/GDSC/gdsc_all_abs_ic50_bayesian_sigmoid_only9dosages.csv', index_col=0)
cell_line_obs_df.index = cell_line_obs_df.index.astype(str)

# cell lines list which your model will be trained on
cell_line_sample_list = cell_line_obs_df.index.astype(str)
len(cell_line_sample_list)

1074

In case you want sample weight based on cancer type. In our example we picked head and neck cell lines.

In [121]:
cell_line_hn_sample_list = tissue_sample_df[tissue_sample_df['TCGA_CLASS']=='HNSC'].index
cell_line_hn_obs_df = cell_line_obs_df.loc[cell_line_hn_sample_list]
len(cell_line_hn_sample_list)

42

## Read drug info

In this example, we focus on 81 drugs that sensitive in head and neck cell lines.

In [96]:
dataset_drug_df = pd.read_csv('../preprocessed_data/GDSC/drug_stat.csv', index_col=0) # hn_drug_stat | drug_stat
dataset_drug_df.index = dataset_drug_df.index.astype(str)

dataset_drug_list = dataset_drug_df.index
dataset_drug_df.shape
print("Dataframe shape:", dataset_drug_df.shape, "\n")
dataset_drug_df.head(5)

Dataframe shape: (226, 27) 



,Drug Name,Synonyms,Target,Target Pathway,Selleckchem Cat#,CAS number,PubCHEM,Others,entropy,max_conc,...,median_ic50_9f,log2_median_ic50_9f,log2_median_ic50_hn,median_ic50_hn,median_ic50_3f_hn,log2_median_ic50_3f_hn,median_ic50_9f_hn,log2_median_ic50_9f_hn,num_sensitive,num_sensitive_hn
Drug ID,,,,,,,,,,,,,,,,,,,,,
1,Erlotinib,"Tarceva, RG-1415, CP-358774, OSI-774, Ro-50823...",EGFR,EGFR signaling,S7786,183321-74-6,176870,"(S1023, 183319-69-9, HCl)",7.045609,2.0,...,8.448890,3.078762,7.764640,217.465095,72.488365,6.179678,24.162788,4.594715,17,1
1001,AICA Ribonucleotide,"AICAR, N1-(b-D-Ribofuranosyl)-5-aminoimidazole...",AMPK agonist,Metabolism,S1802,2627-69-2,65110,NaN,6.034272,2000.0,...,206.748380,7.691732,9.939784,982.139588,327.379863,8.354822,109.126621,6.769859,476,27
1003,Camptothecin,"7-Ethyl-10-Hydroxy-Camptothecin, SN-38, Irinot...",TOP1,DNA replication,S1288,7689-03-4,104842,"(SN-38, S4908, 86639-52-3) (Irinotecan, S1198,...",4.609530,0.1,...,0.002003,-8.963413,-7.587491,0.005199,0.001733,-9.172454,0.000578,-10.757416,688,30
1004,Vinblastine,Velban,Microtubule destabiliser,Mitosis,S1248,143-67-9,6710780,NaN,4.297122,0.1,...,0.001599,-9.289051,-7.150982,0.007036,0.002345,-8.735945,0.000782,-10.320907,753,33
1005,Cisplatin,"cis-Diammineplatinum(II) dichloride, Platinol,...",DNA crosslinker,DNA replication,S1166,15663-27-1,84691,NaN,7.203618,6.0,...,1.328214,0.409488,3.486279,11.206619,3.735540,1.901317,1.245180,0.316354,175,9


## Read gene expression

The file can be download from https://www.dropbox.com/s/3v576mspw5yewbm/GDSC_exp.tsv?dl=0

In [97]:
gene_exp_df = pd.read_csv('../data/GDSC/GDSC_exp.tsv', sep='\t', index_col=0)
print("Dataframe shape:", gene_exp_df.shape, "\n")
gene_exp_df.columns

Dataframe shape: (17737, 1018) 



Index(['906826', '687983', '910927', '1240138', '1240139', '906792', '910688',
       '1240135', '1290812', '907045',
       ...
       '753584', '907044', '998184', '908145', '1659787', '1298157', '1480372',
       '1298533', '930299', '905954.1'],
      dtype='object', length=1018)

If there is any gene with mutiple probes, calculate the mean.

In [98]:
gene_exp_df = gene_exp_df.groupby(gene_exp_df.index).mean()
print("Dataframe shape:", gene_exp_df.shape, "\n")
gene_exp_df.items

Dataframe shape: (17419, 1018) 



<bound method DataFrame.items of             906826    687983    910927   1240138   1240139    906792  \
GENE                                                                   
A1BG      6.208447  5.025810  5.506955  4.208349  3.399366  4.917872   
A1CF      2.981775  2.947547  2.872071  3.075478  2.853231  3.221491   
A2M       3.133883  3.335711  3.287678  3.029602  3.263129  6.877324   
A2ML1     2.652527  2.734617  3.946546  2.776204  2.708683  2.907140   
A3GALT2P  2.759164  2.867167  3.007847  2.628118  2.690651  2.587249   
...            ...       ...       ...       ...       ...       ...   
ZYG11A    3.356583  3.569947  2.777126  2.658556  2.728445  2.862462   
ZYG11B    5.057855  5.841296  4.762718  4.332353  5.469322  4.453835   
ZYX       4.415610  3.985445  4.792370  5.744011  5.402432  4.186831   
ZZEF1     4.406069  5.068493  4.825940  5.190881  5.329415  4.900134   
ZZZ3      8.254065  7.250144  7.696257  6.025487  6.474561  7.503679   

            910688   1240135  

### Normalize gene expression
We normalized baseline gene expression values for each gene by computing fold-changes compared to the average value across cell-lines

In [99]:
cell_line_log2_mean_fc_exp_df, cell_line_mean_exp_df = pp.gexp.normalize_log2_mean_fc(gene_exp_df)
cell_line_log2_mean_fc_exp_df.head(5)

,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,907045,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954.1
GENE,,,,,,,,,,,,,,,,,,,,,
A1BG,1.943238,0.760601,1.241746,-0.056860,-0.865843,0.652663,-0.437121,0.881693,-1.157667,0.796857,...,0.006963,-0.830184,0.664843,-1.364996,0.258502,0.809742,-1.308057,-1.175581,-0.217845,1.064315
A1CF,-0.268618,-0.302846,-0.378322,-0.174915,-0.397162,-0.028902,-0.254037,-0.356416,-0.494724,-0.264743,...,-0.308733,-0.094857,-0.266774,-0.132081,-0.274984,-0.344589,-0.305905,-0.470389,-0.379573,-0.324040
A2M,-0.828135,-0.626306,-0.674340,-0.932415,-0.698889,2.915306,-0.660058,-0.590080,-0.639561,-0.432366,...,-0.074692,-0.489049,-0.716109,2.375523,-0.433984,-0.807035,-0.765430,-0.485882,1.197925,4.888548
A2ML1,-0.294075,-0.211986,0.999944,-0.170398,-0.237920,-0.039463,-0.161534,-0.123814,-0.216341,0.024310,...,-0.048223,-0.159094,-0.238925,-0.116490,-0.189943,-0.201452,-0.040507,-0.032430,-0.158200,0.038056
A3GALT2P,-0.069974,0.038030,0.178710,-0.201020,-0.138486,-0.241889,0.093563,-0.091846,0.165046,0.210778,...,0.129311,0.229658,-0.065662,-0.228245,-0.056884,0.010012,-0.131407,-0.172311,-0.069101,-0.231117


### Read essential genes list

Or in case you want your training using one specific set of genes.

In [117]:
# ess_gene_list = utility.get_gene_list('../data/essential_genes.txt')
ess_gene_list = utility.read_first_column('../data/IntOGen-DriverGenes.tsv')
print(len(ess_gene_list))

634


### Sample with both expression and response data

In [124]:
cell_line_sample_list = np.array([s for s in cell_line_sample_list if s in gene_exp_df.columns])
len(cell_line_sample_list)

985

Arrange gene expression and drug response matrix

In [ ]:
cell_line_log2_mean_fc_exp_df = cell_line_log2_mean_fc_exp_df[cell_line_sample_list]
print(cell_line_obs_df.shape)
cell_line_obs_df = cell_line_obs_df.loc[cell_line_sample_list, dataset_drug_list]
dataset_drug_df = dataset_drug_df.loc[dataset_drug_list]

cell_line_log2_mean_fc_exp_df.shape, cell_line_obs_df.shape, dataset_drug_df.shape

((17419, 985), (985, 226), (226, 27))

# Calculate kernel feature 

Based on all given cell line samples with gene expression profiles and a list of genes (e.g. essential gene list). This step might take a bit more time than the usual.

In [103]:
kernel_feature_df = pp.gexp.calculate_kernel_feature(cell_line_log2_mean_fc_exp_df, cell_line_log2_mean_fc_exp_df, ess_gene_list).loc[cell_line_sample_list]

Calculating kernel features based on 591 common genes
(17419, 985) (17419, 985)
100 of 985 (73.48)s
200 of 985 (74.07)s
300 of 985 (74.29)s
400 of 985 (74.01)s
500 of 985 (70.28)s
600 of 985 (70.25)s
700 of 985 (70.16)s
800 of 985 (70.27)s
900 of 985 (70.32)s


# Model training

In [116]:
# kernel feature based only on training samples
X_train = kernel_feature_df.loc[cell_line_sample_list, cell_line_sample_list]
# observed drug response
Y_train = cell_line_obs_df.loc[cell_line_sample_list]

985


In [105]:
print("Dataframe shape:", X_train.shape, "\n")
X_train.head(2)

Dataframe shape: (985, 985) 



,1240121,1240122,1240123,1240124,1240125,1240127,1240128,1240129,1240130,1240131,...,949175,949176,949177,949178,949179,971773,971774,971777,998184,998189
1240121,1.000000,0.630082,0.112665,0.319995,-0.059488,-0.118530,0.020950,0.164217,0.033459,0.086305,...,-0.416072,-0.319743,-0.377288,-0.376035,-0.310759,-0.146796,-0.255095,-0.180890,-0.283963,0.120126
1240122,0.630082,1.000000,0.164760,0.214418,-0.175414,0.032908,-0.074793,0.142435,-0.002246,0.027819,...,-0.261070,-0.187578,-0.215189,-0.205727,-0.220808,-0.084657,-0.097722,-0.067442,-0.153784,0.123865


In [106]:
print("Dataframe shape:", Y_train.shape, "\n")
Y_train.head(2)

Dataframe shape: (985, 226) 



Drug ID,1,1001,1003,1004,1005,1006,1007,1008,1009,1010,...,64,71,83,86,87,88,89,9,91,94
1240121,NaN,8.922340,-9.065271,-8.301324,1.998429,-2.455685,-11.688212,4.329096,9.033478,-0.035476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1240122,NaN,9.594524,-8.343748,-7.554691,6.033703,-2.145146,-10.332062,4.108168,7.775115,-4.102308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Select CaDRReS training for different objective functions

1. `cadrres-wo-sample-bias`: CaDRReS + no bp (bp = sample bias)
2. `cadrres-wo-sample-bias-weight`: CaDRReS + no bp + ciu + du (ciu = drug-sample weight w.r.t. maximum dosage, du = indication-specific weight). This is **CaDRReS-Sc** model.

In [107]:
obj_function = widgets.Dropdown(options=['cadrres-wo-sample-bias', 'cadrres-wo-sample-bias-weight'], description='Objetice function')

In [108]:
display(obj_function)

Dropdown(description='Objetice function', options=('cadrres-wo-sample-bias', 'cadrres-wo-sample-bias-weight'),…

In [109]:
model_spec_name = obj_function.value # cadrres-wo-sample-bias | cadrres-wo-sample-bias-weight

indication_specific_degree = 1 # multiply by 1 = no indication-specific
# indication_specific_degree = 10

indication_specific_degree

1

## Specify output directory

In [110]:
output_dir = '../my_models/'

In [111]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print ('Results will be saved in ', output_dir)

Results will be saved in  ../my_models/


## Train CaDRReS Model 

Prepare x0 for calculating logistic sample weigh (o_i) based on maximum drug dosage

In [112]:
sample_weights_logistic_x0_df = model.get_sample_weights_logistic_x0(dataset_drug_df, 'log2_max_conc', X_train.index)

Prepare indication weight (skip for this analysis = set all to 1)

In [113]:
indication_weight_df = pd.DataFrame(np.ones(Y_train.shape), index=Y_train.index, columns=Y_train.columns)
cv_cell_line_hn_sample_list = [cl for cl in cell_line_hn_sample_list if cl in X_train.index]
indication_weight_df.loc[cv_cell_line_hn_sample_list, :] = indication_weight_df.loc[cv_cell_line_hn_sample_list, :] * indication_specific_degree

Start model training

In [114]:

if model_spec_name in ['cadrres', 'cadrres-wo-sample-bias']:
    cadrres_model_dict, cadrres_output_dict = model.train_model(Y_train, X_train, Y_train, X_train, 10, 0.0, 100000, 0.01, model_spec_name=model_spec_name, save_interval=5000, output_dir=output_dir)
elif model_spec_name in ['cadrres-wo-sample-bias-weight']:
    cadrres_model_dict, cadrres_output_dict = model.train_model_logistic_weight(Y_train, X_train, Y_train, X_train, sample_weights_logistic_x0_df, indication_weight_df, 10, 0.0, 100000, 0.01, model_spec_name=model_spec_name, save_interval=5000, output_dir=output_dir)

Initializing the model...
Building the TensorFlow model...
Training the model...
Step 0, Train Loss: 18.2567, Time Elapsed: 0.01 min
Step 5000, Train Loss: 12.1967, Time Elapsed: 0.69 min
Step 10000, Train Loss: 9.2043, Time Elapsed: 1.37 min
Step 15000, Train Loss: 7.3517, Time Elapsed: 2.05 min
Step 20000, Train Loss: 6.1841, Time Elapsed: 2.77 min
Step 25000, Train Loss: 5.4320, Time Elapsed: 3.54 min
Step 30000, Train Loss: 4.9395, Time Elapsed: 4.31 min
Step 35000, Train Loss: 4.6127, Time Elapsed: 5.06 min
Step 40000, Train Loss: 4.3917, Time Elapsed: 5.80 min
Step 45000, Train Loss: 4.2385, Time Elapsed: 6.56 min
Step 50000, Train Loss: 4.1291, Time Elapsed: 7.32 min
Step 55000, Train Loss: 4.0482, Time Elapsed: 7.97 min
Step 60000, Train Loss: 3.9863, Time Elapsed: 8.61 min
Step 65000, Train Loss: 3.9374, Time Elapsed: 9.26 min
Step 70000, Train Loss: 3.8974, Time Elapsed: 9.90 min
Step 75000, Train Loss: 3.8637, Time Elapsed: 10.55 min
Step 80000, Train Loss: 3.8348, Time Elap

## Save the final CaDRReS Model trained on your dataset

- cadrres_model_dict contains model hyperparameters and trained parameters
- cadrres_output_dict contains data and prediction on training dataset

In [115]:
print('Saving ' + output_dir + '{}_param_dict.pickle'.format(model_spec_name))
pickle.dump(cadrres_model_dict, open(output_dir + '{}_param_dict.pickle'.format(model_spec_name), 'wb'))
print('Saving ' + output_dir + '{}_output_dict.pickle'.format(model_spec_name))
pickle.dump(cadrres_output_dict, open(output_dir + '{}_output_dict.pickle'.format(model_spec_name), 'wb'))

Saving ../my_models/cadrres-wo-sample-bias_param_dict.pickle
Saving ../my_models/cadrres-wo-sample-bias_output_dict.pickle


# Next Step
After you saved the CaDRReS-Sc model here, you can follow this [tutorial](./prediction_pretrained_model.ipynb) to predict drug response based on the pre-trained model.

---

**Authors:** [Chayaporn Suphavilai](mailto:@.com), [Rafael Peres da Silva](), Genome Institute of Singapore, Nagarajan Lab, April 30, 2020

---

Reproducibility tips from https://github.com/jupyter-guide/ten-rules-jupyter